In [ ]:
import pandas as pd

In [ ]:
# change the file paths
#Rating Google Play
file_path = '/xxx.csv'
#Crashes Google Play
file_path2 = '/xxx.csv'
#Uninstalls Google Play
file_path3 = '/xxx.csv'
#Installs AppsFlyer
file_path4 = '/xxx.csv'
#IAP US BigQuery
file_path5 = '/xxx.csv'
#IAP non-USA BigQuery
file_path6 = '/xxx.csv'
#Sessions non-USA BigQuery
file_path7 = '/xxx.csv'
#Sessions USA BigQuery
file_path8 = '/xxx.csv'


In [ ]:
with open(file_path, encoding="utf16", errors='ignore') as file:
    df = pd.read_csv(file)

df.head()

In [ ]:
df['Daily Average Rating'] = df['Daily Average Rating'].mask(df['Daily Average Rating'] == 0, pd.NA)

df['Country'] = df['Country'].replace('United States', 'US')

df['Country Category'] = df['Country'].apply(lambda x: 'US' if x == 'US' else 'non-USA')

result_df = df.groupby(['Date', 'Country Category'], as_index=False)['Daily Average Rating'].mean()

# Pivot table
df1 = pd.pivot_table(result_df, values='Daily Average Rating', index='Date',
                       columns=['Country Category'], aggfunc="mean")

print(df1)

In [ ]:
with open(file_path2, encoding="utf16", errors='ignore') as file:
    df2 = pd.read_csv(file)

df2.head()

In [ ]:
df2.set_index(['Date'])

In [ ]:
df2[['Daily Crashes', 'Daily ANRs']] /= 2
df2 = df2.rename(columns={"Daily Crashes": "Daily Crashes US", "Daily ANRs": "Daily ANRs US"})

In [ ]:
df2.head()

In [ ]:
df2['Daily Crashes non-USA'] = df2.loc[:, 'Daily Crashes US']
df2['Daily ANRs non-USA'] = df2.loc[:, 'Daily ANRs US']
df2.drop(['Package Name'], axis=1, inplace=True)

In [ ]:
df2.head()

In [ ]:
df_merged2 = pd.merge(df1, df2, on='Date')

df_merged2 = df_merged2.dropna()

df_merged2 = df_merged2.rename(columns={"US": "Daily Rating US", "non-USA": "Daily Rating non-USA"})

df_merged2

In [ ]:
with open(file_path3, encoding="utf16", errors='ignore') as file:
    df3 = pd.read_csv(file)

df3.head()

In [ ]:
df3['Daily User Uninstalls'] = df3['Daily User Uninstalls'].mask(df3['Daily User Uninstalls'] == 0, pd.NA)

df3['Country'] = df3['Country'].replace('United States', 'US')

df3['Country Category'] = df3['Country'].apply(lambda x: 'Uninstalls US' if x == 'US' else 'Uninstalls non-USA')

result_df3 = df3.groupby(['Date', 'Country Category'], as_index=False)['Daily User Uninstalls'].sum()

# Pivot table
df3 = pd.pivot_table(result_df3, values='Daily User Uninstalls', index='Date',
                       columns=['Country Category'], aggfunc="sum")

print(df3)

In [ ]:
df_merged3 = pd.merge(df_merged2, df3, on='Date')

df_merged3 = df_merged3.dropna()

df_merged3

In [ ]:
with open(file_path4, encoding="utf8", errors='ignore') as file:
    df4 = pd.read_csv(file)

In [ ]:
df4['Installs'] = df4['Installs'].replace(',', '', regex=True).astype(float)
df4

In [ ]:
df4['Geo'] = df4['Geo'].replace('United States (US)', 'US')

df4['Country Category'] = df4['Geo'].apply(lambda x: 'Installs US' if x == 'US' else 'Installs non-USA')

result_df4 = df4.groupby(['Date', 'Country Category'], as_index=False)['Installs'].sum()

# Pivot table
df4 = pd.pivot_table(result_df4, values='Installs', index='Date',
                       columns=['Country Category'], aggfunc="sum")

print(df4)

In [ ]:
df_merged4 = pd.merge(df_merged3, df4, how='right', on='Date')
df_merged4

In [ ]:
with open(file_path5, encoding="utf8", errors='ignore') as file:
    df5 = pd.read_csv(file)

# comment out the code below if you selected the columns as self-explanatory already
df5.rename(columns={'f0_': 'IAP US', 'dt': 'Date'}, inplace=True)
df5

In [ ]:
with open(file_path6, encoding="utf8", errors='ignore') as file:
    df6 = pd.read_csv(file)

df6.rename(columns={'f0_': 'IAP non-USA', 'dt': 'Date'}, inplace=True)
df6

In [ ]:
df_merged5 = pd.merge(df5, df6, on='Date')
df_merged5

In [ ]:
df_merged6 = pd.merge(df_merged5, df_merged4, how = 'right', on='Date')
df_merged6

In [ ]:
with open(file_path7, encoding="utf8", errors='ignore') as file:
    df7= pd.read_csv(file)

with open(file_path8, encoding="utf8", errors='ignore') as file:
    df8 = pd.read_csv(file)

In [ ]:
df_merged7 = pd.merge(df7, df8, on='Date')
df_merged7

In [ ]:
df_merged8 = pd.merge(df_merged6, df_merged7, how = 'right', on='Date')
df_merged8

In [ ]:
df_merged8['Crash Rate US'] = (df_merged8['Daily Crashes US'] / df_merged8['Sessions_USA']) * 100
df_merged8['Crash Rate non-USA'] = (df_merged8['Daily Crashes non-USA'] / df_merged8['Sessions_nonUSA']) * 100
df_merged8['ANR Rate US'] = (df_merged8['Daily ANRs US'] / df_merged8['Sessions_USA']) * 100
df_merged8['ANR Rate non-USA'] = (df_merged8['Daily ANRs non-USA'] / df_merged8['Sessions_nonUSA']) * 100


df_merged8